In [1]:
import os
import sys
import glob
import numpy as np
import pandas as pd
from IPython.display import display
import time 
import tracker
latest_date = tracker.post_hicpro_processing_dates[-1]
os.chdir('/mnt/BioAdHoc/Groups/vd-ay/kfetter/hichip-db-loop-calling/')
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 1000)
pd.set_option('max_colwidth', 400)

In [2]:
latest_date

'2022.07.12.09.12'

In [3]:
# setting input and output with jupyter notebook in context 
if 'ipykernel_launcher.py' in sys.argv[0]:
    input_fn = 'results/samplesheets/post-hicpro/{}.post-hicpro.samplesheet.with_header.tsv'.format(latest_date)
    output_prefix = 'results/samplesheets/post-hicpro/{}.post-hicpro.post-check.fithichip-peaks'.format(latest_date)
else:
    input_fn = sys.argv[1]
    output_fn = sys.argv[2]

## Helper Functions

In [4]:
def read_out_log(log):
    
    status = 0
    sample_name = 'check'
    date = ''

    # extract job_id and index
    meta = os.path.basename(log)
    meta = meta.split('.')[1].split('-')
    job_id = meta[0].replace('o', '')
    index = meta[1]

    with open(log, errors='ignore') as fr:
        for line in fr:
            info = line.strip()
            if 'sample_name' in info:
                sample_name = info.split()[1]
            elif '# Ended: hichip-peaks' in info:
                status = 1

    return([index, sample_name, job_id, status])

In [5]:
def read_error_log(log):
    
    status = 0
    
    with open(log, errors='ignore') as fr:
        for line in fr:
            info = line.strip()
            if 'INFO - Done!' in info:
                status = 1
    
    return(status)

## Check Logs

In [6]:
# load the data
df = pd.read_table(input_fn)
df['sample_index'] = df.index + 1 # add 1 since we'll be using 1-based indexing with sed

In [7]:
df

,std_sample_name,gse_id,organism,bio_rep,antibody_target,restriction_enzyme,sample_name,sample_index
0,293T.GSE128106.Homo_Sapiens.YY1.b1,GSE128106,Homo_Sapiens,1,YY1,HindIII,293T,1
1,293T-PDS.GSE128106.Homo_Sapiens.YY1.b1,GSE128106,Homo_Sapiens,1,YY1,HindIII,293T-PDS,2
2,293T-TMPYP4.GSE128106.Homo_Sapiens.YY1.b1,GSE128106,Homo_Sapiens,1,YY1,HindIII,293T-TMPYP4,3
3,A673_SA1m1.GSE133227.Homo_Sapiens.CTCF.b1,GSE133227,Homo_Sapiens,1,CTCF,MboI,A673_SA1m1,4
4,A673_SA1m1.GSE133227.Homo_Sapiens.CTCF.b2,GSE133227,Homo_Sapiens,2,CTCF,MboI,A673_SA1m1,5
5,A673_SA2m1.GSE133227.Homo_Sapiens.CTCF.b1,GSE133227,Homo_Sapiens,1,CTCF,MboI,A673_SA2m1,6
6,A673_SA2m1.GSE133227.Homo_Sapiens.CTCF.b2,GSE133227,Homo_Sapiens,2,CTCF,MboI,A673_SA2m1,7
7,A673_SA2m1.GSE133227.Homo_Sapiens.CTCF.b3,GSE133227,Homo_Sapiens,3,CTCF,MboI,A673_SA2m1,8
8,A673_SA2m1.GSE133227.Homo_Sapiens.CTCF.b4,GSE133227,Homo_Sapiens,4,CTCF,MboI,A673_SA2m1,9
9,A673_SA2m1.GSE133227.Homo_Sapiens.CTCF.b5,GSE133227,Homo_Sapiens,5,CTCF,MboI,A673_SA2m1,10


In [8]:
print('number of samples:', len(df))

number of samples: 338


In [9]:
# create a dataframe of log information
output_logs = glob.glob('results/peaks/logs/run_hichip_peaks.o*-*')
log_data = []   
for out_log in output_logs: 
    sample_index = out_log.split('-')[1]
    index, sample_name, job_id, out_status = read_out_log(out_log)
    error_log = [log for log in glob.glob('results/peaks/logs/run_hichip_peaks.e*-*') if log.split('-')[1] == sample_index][0]
    error_status = read_error_log(error_log)
    log_data.append([index, sample_name, job_id, out_status, error_status, os.path.basename(out_log)])
log_df = pd.DataFrame(log_data)
log_df.columns = ['sample_index', 'std_sample_name', 'job_id', 'out_status', 'error_status', 'log']
log_df.drop('sample_index', inplace=True, axis=1)

In [10]:
log_df = log_df.sort_values(by=["std_sample_name"], ascending=True).reset_index(drop=True)
log_df

,std_sample_name,job_id,out_status,error_status,log
0,293T-PDS.GSE128106.Homo_Sapiens.YY1.b1,5049151,1,1,run_hichip_peaks.o5049151-2
1,293T-TMPYP4.GSE128106.Homo_Sapiens.YY1.b1,5049151,1,1,run_hichip_peaks.o5049151-3
2,293T.GSE128106.Homo_Sapiens.YY1.b1,5048890,1,1,run_hichip_peaks.o5048890-1
3,A673-siCT-Dh1-72h-D347-D362.GSE156650.Homo_Sapiens.CTCF.b1,5049151,1,1,run_hichip_peaks.o5049151-31
4,A673-siCT-Dh1-72h-D347-D362.GSE156650.Homo_Sapiens.CTCF.b2,5049151,1,1,run_hichip_peaks.o5049151-32
5,A673-siSA2-Dh6-72h-D347-D362.GSE156650.Homo_Sapiens.CTCF.b1,5049151,1,1,run_hichip_peaks.o5049151-33
6,A673-siSA2-Dh6-72h-D347-D362.GSE156650.Homo_Sapiens.CTCF.b2,5049151,1,1,run_hichip_peaks.o5049151-34
7,A673-siSA2-Dh8-72h-D347-D362.GSE156650.Homo_Sapiens.CTCF.b1,5049151,1,1,run_hichip_peaks.o5049151-35
8,A673-siSA2-Dh8-72h-D347-D362.GSE156650.Homo_Sapiens.CTCF.b2,5049151,1,1,run_hichip_peaks.o5049151-36
9,A673_SA1m1.GSE133227.Homo_Sapiens.CTCF.b1,5049151,1,1,run_hichip_peaks.o5049151-4


## Identifying problem samples

In [11]:
problems = (log_df.out_status != 1) | (log_df.error_status != 1)
problems_df = log_df.loc[problems,:]
print('nubmer of problem samples:', len(problems_df))

nubmer of problem samples: 9


In [12]:
problems_df.loc[:, ['std_sample_name', 'job_id', 'out_status', 'out_status', 'log']]

,std_sample_name,job_id,out_status,out_status,log
153,KYSE180.GSE155184.Homo_Sapiens.H3K27ac.b1,5049151,0,0,run_hichip_peaks.o5049151-187
259,T47D-T0.GSE179666.Homo_Sapiens.PR.b1,5049151,0,0,run_hichip_peaks.o5049151-293
260,T47D-T30.GSE179666.Homo_Sapiens.PR.b1,5049151,0,0,run_hichip_peaks.o5049151-294
299,VCaP-AU-4h.GSE171591.Homo_Sapiens.CTCF.b1,5049151,0,0,run_hichip_peaks.o5049151-332
300,VCaP-AU-4h.GSE171591.Homo_Sapiens.H3K27ac.b1,5049151,0,0,run_hichip_peaks.o5049151-331
301,VCaP-AU-4h.GSE171591.Homo_Sapiens.H3K4me4.b1,5049151,0,0,run_hichip_peaks.o5049151-330
302,VCaP-DMSO-4h.GSE171591.Homo_Sapiens.CTCF.b1,5049151,0,0,run_hichip_peaks.o5049151-337
303,VCaP-DMSO-4h.GSE171591.Homo_Sapiens.H3K27ac.b1,5049151,0,0,run_hichip_peaks.o5049151-336
304,VCaP-DMSO-4h.GSE171591.Homo_Sapiens.H3K4me3.b1,5049151,0,0,run_hichip_peaks.o5049151-335


In [17]:
header_output = '{}.with_header.tsv'.format(output_prefix)
problems_df.to_csv(header_output, header=True, index=False, sep='\t')